In [1]:
from pyspark.sql import SparkSession

"""make an instance of a SparkSession called 'spark'."""
spark = SparkSession.builder.master('local').getOrCreate()


22/06/24 12:03:44 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/06/24 12:03:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/24 12:03:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
"""Project Setup:"""
# In a main.ipynb file, make an instance of a SparkSession called 'spark'.
# In the header of the data, give the index column a name. Otherwise, you'll get the error "CSVHeaderChecker: CSV header does not conform to the schema".
# Read the Spotify artists CSV file into a Spark DataFrame with the following snippet:
# spark_df = (spark.read.format("csv").options(header="true").load("../project-1/data/spotify_artists.csv"))


In [13]:
import os

# Read the Spotify artists CSV file into a Spark DataFrame
spark_df = (spark.read.format("csv").options(header="true").load("data/spotify_artists.csv"))


In [14]:
"""Profile the Data:"""
# Show a description (summary) of the Spark DataFrame.
spark_df.describe


<bound method DataFrame.describe of DataFrame[_c0: string, artist_popularity: string, followers: string, genres: string, id: string, name: string, track_id: string, track_name_prev: string, type: string]>

In [21]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

custom_schema = StructType() \
    StructField('index', IntegerType, true),
    StructField('artist_popularity', IntegerType, true),
    StructField('followers', IntegerType, true),
    StructField('genres', StringType, true),
    StructField('id', StringType, true),
    StructField('name', StringType, true),
    StructField('track_id', StringType, true),
    StructField('track_name_prev', StringType, true),
    StructField('type', StringType, true)


SyntaxError: invalid syntax (3717256039.py, line 4)

In [15]:
# Print the schema of the DataFrame.
spark_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- artist_popularity: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name_prev: string (nullable = true)
 |-- type: string (nullable = true)



In [18]:
# Select and show just the first 10 values in the 'name' and 'genres' columns.
spark_df.select(spark_df.name, spark_df.genres).show(10)


NameError: name 'name' is not defined